In [39]:
import requests
import xml.etree.ElementTree as ET
from datetime import datetime


In [40]:
def search_arxiv_papers(query, max_results=3):
    base_url = "http://export.arxiv.org/api/query?"
    search_query = f"search_query=all:{query}&start=0&max_results={max_results}&sortBy=submittedDate&sortOrder=descending"
    response = requests.get(base_url + search_query)

    root = ET.fromstring(response.content)

    papers = []
    for entry in root.findall('{http://www.w3.org/2005/Atom}entry'):
        title = entry.find('{http://www.w3.org/2005/Atom}title').text.strip()
        summary = entry.find('{http://www.w3.org/2005/Atom}summary').text.strip()
        published = entry.find('{http://www.w3.org/2005/Atom}published').text
        link = entry.find('{http://www.w3.org/2005/Atom}id').text

        papers.append({
            "title": title,
            "summary": summary,
            "published": published,
            "link": link
        })

    return papers


In [41]:
def search_arxiv_papers(max_results=3):
    base_url = "http://export.arxiv.org/api/query?"

    # Broad, realistic agriculture-related AI query
    search_query = (
        "search_query=("
        "all:agriculture OR "
        "all:crop OR "
        "all:plant OR "
        "all:farming OR "
        "all:yield OR "
        "all:remote+sensing"
        ")"
        "&start=0"
        f"&max_results={max_results}"
        "&sortBy=submittedDate"
        "&sortOrder=descending"
    )

    url = base_url + search_query
    response = requests.get(url)

    root = ET.fromstring(response.content)

    papers = []
    for entry in root.findall('{http://www.w3.org/2005/Atom}entry'):
        papers.append({
            "title": entry.find('{http://www.w3.org/2005/Atom}title').text.strip(),
            "summary": entry.find('{http://www.w3.org/2005/Atom}summary').text.strip(),
            "published": entry.find('{http://www.w3.org/2005/Atom}published').text,
            "link": entry.find('{http://www.w3.org/2005/Atom}id').text
        })

    return papers



In [42]:
def filter_agriculture_papers(papers):
    keywords = [
        "agriculture", "crop", "farming", "soil",
        "plant", "yield", "irrigation", "harvest",
        "remote sensing", "phenotyping"
    ]

    relevant = []
    for paper in papers:
        text = (paper["title"] + " " + paper["summary"]).lower()
        score = sum(1 for keyword in keywords if keyword in text)

        if score > 0:
            paper["relevance_score"] = score
            relevant.append(paper)

    # If nothing matches strictly, fallback to top papers
    if not relevant:
        return papers[:3]

    # Sort by relevance
    relevant.sort(key=lambda x: x["relevance_score"], reverse=True)
    return relevant[:3]



In [43]:
def summarize_text(text, max_sentences=3):
    sentences = text.split('. ')
    summary = '. '.join(sentences[:max_sentences])
    return summary.strip() + '.'


In [44]:
for paper in papers:
    paper["summary"] = summarize_text(paper["summary"])

In [45]:
def store_results(papers):
    memory = []
    for p in papers:
        memory.append({
            "title": p["title"],
            "published_date": p["published"],
            "summary": p["summary"],
            "link": p["link"]
        })
    return memory

In [46]:
def agent_execute(user_goal):
    plan = [
        "Search recent AI research papers related to agriculture",
        "Summarize the research papers",
        "Store results in structured format"
    ]

    print("AGENT PLAN:")
    for step in plan:
        print("-", step)

    papers = search_arxiv_papers(3)

    for paper in papers:
        paper["summary"] = summarize_text(paper["summary"])

    memory = store_results(papers)
    return memory


In [47]:
user_goal = "Find the top 3 recent AI research papers on agriculture, summarize them, and store the output in a structured format."

final_output = agent_execute(user_goal)
final_output


AGENT PLAN:
- Search recent AI research papers related to agriculture
- Summarize the research papers
- Store results in structured format


[{'title': 'DyTopo: Dynamic Topology Routing for Multi-Agent Reasoning via Semantic Matching',
  'published_date': '2026-02-05T18:59:51Z',
  'summary': "Multi-agent systems built from prompted large language models can improve multi-round reasoning, yet most existing pipelines rely on fixed, trajectory-wide communication patterns that are poorly matched to the stage-dependent needs of iterative problem solving. We introduce DyTopo, a manager-guided multi-agent framework that reconstructs a sparse directed communication graph at each round. Conditioned on the manager's round goal, each agent outputs lightweight natural-language query (need) and \\key (offer) descriptors; DyTopo embeds these descriptors and performs semantic matching, routing private messages only along the induced edges.",
  'link': 'http://arxiv.org/abs/2602.06039v1'},
 {'title': 'InterPrior: Scaling Generative Control for Physics-Based Human-Object Interactions',
  'published_date': '2026-02-05T18:59:27Z',
  'summary'